In [14]:
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader #works with PDF, txt, docx
#we don't need to care about the type of the files
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

cache_dir = LocalFileStore("./.cache2/")

llm = ChatOpenAI(
    temperature=0.1
)

embedder = OpenAIEmbeddings()

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size = 600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up: \n\n{context}")
    ,
    ("human", "{question}")
])

chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

#chain.invoke("Who is Julia?")

memory = ConversationBufferMemory()

question_1 = "Is Aaronson guilty?"
output_1 = str(chain.invoke(question_1))
# # will be saved to the memory

memory.save_context({"input": question_1}, {"output": output_1})

question_2 = "What message did he write on the table?"
output_2 = str(chain.invoke(question_2))
# # # will be saved to the memory

memory.save_context({"input": question_2}, {"output": output_2})

question_3 = "Who is Julia?"
output_3 =  str(chain.invoke(question_3))
# # # will be saved to the memory

memory.save_context({"input": question_3}, {"output": output_3})

memory.load_memory_variables({})


{'history': 'Human: Is Aaronson guilty?\nAI: content="I don\'t know."\nHuman: What message did he write on the table?\nAI: content=\'He wrote: "FREEDOM IS SLAVERY" and then "TWO AND TWO MAKE FIVE" on the table.\'\nHuman: Who is Julia?\nAI: content=\'Julia is a character mentioned in the text who seems to be someone the protagonist loves and feels a strong connection to.\''}